This sets up the PCA based analysis

### Import necessary packages and tools

In [ ]:
#@title Imports

from google.cloud import storage

import time

import pandas as pd
import numpy as np
import geopandas as gpd

from datetime import datetime, timedelta

import os

In [4]:
#@title Plotting
import plotly
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots

# Analysis
from scipy.stats import pearsonr

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error as get_rmse
from sklearn.model_selection import RandomizedSearchCV, cross_val_predict

from xgboost import XGBRegressor

In [ ]:
#@title Constants and misc. helpers

# Data directories
os.chdir('/Users/viveksakhrani/Documents/GitHub/lacuna-ai4ea-INT')
DATA_DIR = os.getcwd()
GS_DATA_DIR = 'gs://ads-lacuna-ai4ea/data'

#FIG_DIR = 'drive/MyDrive/Lacuna/figs/%s/' % datetime.today().strftime('%Y%m')
#! mkdir -p $FIG_DIR

# Other constants
CONTINUOUS_VARS = ['tot_population', 'med_temperature', 'med_precipitation',
                   'med_awi', 'med_spending', 'electrification_prop_ti']
APPLIANCES = ['Fan', 'Laptop / Computer', 'Light bulb', 'Mobile phone charger',
              'Radio', 'Television']

# Helpers
def get_row_col(i, n_cols):
    return int(np.ceil((i + 1) / n_cols)), i % n_cols + 1

In [18]:
current_dir = DATA_DIR
print(f"Current directory: {current_dir}")

Current directory: /Users/viveksakhrani/Documents/GitHub/lacuna-ai4ea-INT


In [14]:
print(GS_DATA_DIR)
peoplesun_hhapps_anon = pd.read_csv('gs://ads-lacuna-ai4ea/data/dataverse/peoplesun_hhapps_anon.csv')


gs://ads-lacuna-ai4ea/data


_request non-retriable exception: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401
Traceback (most recent call last):
  File "/Users/viveksakhrani/miniconda3/envs/sandbox/lib/python3.12/site-packages/gcsfs/retry.py", line 135, in retry_request
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/viveksakhrani/miniconda3/envs/sandbox/lib/python3.12/site-packages/gcsfs/core.py", line 470, in _request
    validate_response(status, contents, path, args)
  File "/Users/viveksakhrani/miniconda3/envs/sandbox/lib/python3.12/site-packages/gcsfs/retry.py", line 122, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401


HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401

In [6]:
#fts = pd.read_csv(f'{GS_DATA_DIR}/processed_fts.csv')

labels = pd.read_csv(f'{GS_DATA_DIR}/dataverse/peoplesun_hhapps_anon.csv')
geo_data = gpd.read_file(f'{GS_DATA_DIR}/eaidgeokey_hh.geojson')
geo_data = geo_data.assign(point=geo_data['geometry'],
                           geometry=geo_data.to_crs(3857)['geometry'].buffer(2000))

drop_cols = ['zone', 'state', 'lga', 'settlement', '_geo_latitude', '_geo_longitude']
fts = geo_data[['hhid', 'point', 'geometry']].merge(fts).drop(drop_cols, axis=1)

labels.columns = ['hhid', 'appliance', 'n_owned', 'hours_day', 'hours_night']

_request non-retriable exception: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401
Traceback (most recent call last):
  File "/Users/viveksakhrani/miniconda3/envs/sandbox/lib/python3.12/site-packages/gcsfs/retry.py", line 135, in retry_request
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/viveksakhrani/miniconda3/envs/sandbox/lib/python3.12/site-packages/gcsfs/core.py", line 470, in _request
    validate_response(status, contents, path, args)
  File "/Users/viveksakhrani/miniconda3/envs/sandbox/lib/python3.12/site-packages/gcsfs/retry.py", line 122, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401


HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401